In [65]:
import pandas as pd

# # Specify the path to your pickle file
# pickle_filename = 'df_routes.pkl'

# df_routes = pd.read_pickle(pickle_filename)


# Define the file path
nodes_path = r"C:\Users\Doktor\Documents\GitHub\Bridges\Data\Nodes_processed.csv"
roads_path = r"C:\Users\Doktor\Documents\GitHub\Bridges\Data\Roads_processed.csv"
bridges_path = r"C:\Users\Doktor\Documents\GitHub\Bridges\Data\Bridges_processed.csv"
bridges_selected_path = r"C:\Users\Doktor\Documents\GitHub\Bridges\Data\Bridges_selected.csv"
survey_path = r"C:\Users\Doktor\Documents\GitHub\Bridges\Data\surveys\Survey.csv"


# Load the CSV data into a DataFrame
df_nodes = pd.read_csv(nodes_path, encoding='utf-8', delimiter=',')
df_roads = pd.read_csv(roads_path, encoding='utf-8', delimiter=',')
df_bridges_all = pd.read_csv(bridges_path, encoding='utf-8', delimiter=',')
df_bridges_sel = pd.read_csv(
    bridges_selected_path, encoding='utf-8', delimiter=',')
df_survey = pd.read_csv(survey_path, encoding='utf-8', delimiter=',')

C:\Users\Doktor\AppData\Local\Temp\ipykernel_1624\3324112587.py:18: DtypeWarning: Columns (5,16) have mixed types. Specify dtype option on import or set low_memory=False.
  df_nodes = pd.read_csv(nodes_path, encoding='utf-8', delimiter=',')
C:\Users\Doktor\AppData\Local\Temp\ipykernel_1624\3324112587.py:19: DtypeWarning: Columns (10,11,12,25,26,27) have mixed types. Specify dtype option on import or set low_memory=False.
  df_roads = pd.read_csv(roads_path, encoding='utf-8', delimiter=',')


In [66]:
bridge_1_to_replace = 'M5910'
bridge_1_replace_by = 'M6900'
bridge_1_start = '48.684675, 18.524662'
bridge_1_end   = '48.643682, 18.487954'
bridge_1_coor = '48.676146, 18.519047'
bridge_1_survey = '91460'
bridge_1_okres = 'Prievidza'

bridge_2_to_replace = 'M7339'
bridge_2_replace_by = 'M7339'
bridge_2_start = '48.38264, 18.39728'
bridge_2_end = '48.345141, 18.293945'
bridge_2_coor = '48.354953, 18.379679'
bridge_2_survey = '80380'
bridge_2_okres = 'Zlaté Moravce'

bridge_3_to_replace = 'M3069'
bridge_3_replace_by = 'M7361'
bridge_3_start = '48.844674, 18.933007'
bridge_3_end = '48.839538, 18.869712'
bridge_3_coor = '48.838050, 18.879447'
bridge_3_survey = '91500'
bridge_3_okres = 'Turčianske Teplice'

bridge_4_to_replace = 'M3011'
bridge_4_replace_by = 'M3011'
bridge_4_start = '48.170121, 17.073916'
bridge_4_end = '48.154463, 17.106079'
bridge_4_coor = '48.162426, 17.097242'
bridge_4_survey = '80122'
bridge_4_okres = 'Bratislava I'

In [67]:
import pandas as pd

# Create a list of dictionaries with bridge replacement information
bridge_replacements = [
    {
        'ID_mosta': bridge_1_replace_by,
        'Okres': 'Prievidza',
        'br_lat': 48.676146,
        'br_long': 18.519047,
        'start_lat': 48.684675,
        'start_long': 18.524662,
        'end_lat': 48.643682,
        'end_long': 18.487954,
        'survey_ID': '91460',
    },
    {
        'ID_mosta': bridge_2_replace_by,
        'Okres': 'Zlaté Moravce',
        'br_lat': 48.354953,
        'br_long': 18.379679,
        'start_lat': 48.38264,
        'start_long': 18.39728,
        'end_lat': 48.345141,
        'end_long': 18.293945,
        'survey_ID': '80380',
    },
    {
        'ID_mosta': bridge_3_replace_by,
        'Okres': 'Turčianske Teplice',
        'br_lat': 48.838050,
        'br_long': 18.879447,
        'start_lat': 48.844674,
        'start_long': 18.933007,
        'end_lat': 48.839538,
        'end_long': 18.869712,
        'survey_ID': '91500',
    },
    {
        'ID_mosta': bridge_4_replace_by,
        'Okres': 'Bratislava I',
        'br_lat': 48.162426,
        'br_long': 17.097242,
        'start_lat': 48.170121,
        'start_long': 17.073916,
        'end_lat': 48.154463,
        'end_long': 17.106079,
        'survey_ID': '80122',
    }
]

# Create a DataFrame from the list of dictionaries
df_routes = pd.DataFrame(bridge_replacements)

In [68]:
from math import radians, cos, sin, asin, sqrt, pi


def get_hexagon_coordinates(row, radius):
    """
    Calculate the coordinates of a hexagon centered at a given latitude and longitude.

    Parameters:
        row (pandas.Series): A row from the DataFrame containing 'br_lat' and 'br_long' columns.
        radius (float): The radius in meters of the circle in which the hexagon is inscribed.

    Returns:
        list of tuples: A list of tuples containing (latitude, longitude) coordinates of the hexagon vertices.
    """
    coordinates = []
    center_lat = row['br_lat']
    center_lon = row['br_long']
    for i in range(6):
        angle = 2 * pi / 6 * i
        dx = radius * cos(angle)
        dy = radius * sin(angle)
        point_lat = center_lat + (180/pi) * (dy / 6378137)
        point_lon = center_lon + (180/pi) * \
            (dx / 6378137) / cos(center_lat * pi/180)
        coordinates.append((point_lat, point_lon))
    return coordinates


# Radius of the circle in meters in which hexagon is inscribed
radius = 20  # 10 meters radius in kilometers

# Create a new column 'Hexagon_Coordinates' in df_routes
df_routes['Hexagon_Coordinates'] = None  # Initialize with None values

# Iterate through each row in the DataFrame
for index, row in df_routes.iterrows():
    # Create hexagon coordinates for each row
    hexagon_coords = get_hexagon_coordinates(row, radius)
    hexagon_coords.append(hexagon_coords[0])

    # Convert hexagon coordinates from (latitude, longitude) tuples to [longitude, latitude] lists
    hexagon_coords_for_api = [[lon, lat] for lat, lon in hexagon_coords]

    # Assign the hexagon coordinates to the 'Hexagon_Coordinates' column
    df_routes.at[index, 'Hexagon_Coordinates'] = hexagon_coords_for_api

In [69]:
import json
import pandas as pd

# Define the generate_graphhopper_request function without the row_index argument


def generate_graphhopper_request(row: pd.Series, profile: str = "car", multiplier: int = 1) -> str:
    """
    Generate a JSON request for the GraphHopper API based on DataFrame row data.

    Parameters:
        row (pd.Series): A row from the DataFrame containing route data.
        profile (str, optional): The vehicle profile for routing (e.g., "car", "bike", "foot").
        multiplier (int, optional): Multiplier for custom model priority within a specific area.

    Returns:
        str: A JSON string representing the GraphHopper API request.
    """
    hexagon_coords = row['Hexagon_Coordinates']

    data = {
        "points": [
            [row['start_long'], row['start_lat']],
            [row['end_long'], row['end_lat']]
        ],
        "profile": profile,
        "points_encoded": False,
        "ch.disable": True,
        "custom_model": {
            "distance_influence": 15,
            "priority": [
                {
                    "if": "in_area1",
                    "multiply_by": multiplier
                }
            ],
            "speed": [],
            "areas": {
                "area1": {
                    "type": "Feature",
                    "geometry": {
                        "type": "Polygon",
                        "coordinates": [hexagon_coords]
                    },
                    "properties": {}
                }
            }
        }
    }
    # Convert the dictionary to a JSON string
    json_data = json.dumps(data, indent=2)
    return json_data

In [70]:
# Create empty columns 'car_req_1' and 'car_req_alt'
df_routes['car_req_1'] = None
df_routes['car_req_alt'] = None

# Before assigning JSON strings, make sure the columns are of type 'object'
df_routes['car_req_1'] = df_routes['car_req_1'].astype(object)
df_routes['car_req_alt'] = df_routes['car_req_alt'].astype(object)

# Iterate through each bridge in the DataFrame
for index, row in df_routes.iterrows():
    # Generate JSON requests for the current bridge and add them directly to the DataFrame
    df_routes.at[index, 'car_req_1'] = generate_graphhopper_request(
        row, profile="car", multiplier=1)
    df_routes.at[index, 'car_req_alt'] = generate_graphhopper_request(
        row, profile="car", multiplier=0)

In [71]:
# Create empty columns 'car_resp_1' and 'car_resp_alt'
df_routes['car_resp_1'] = None
df_routes['car_resp_alt'] = None

In [72]:
import requests
import json


def call_graphhopper_api(df, row_index, api_key, verbose=False):
    api_endpoint = "https://graphhopper.com/api/1/route"
    headers = {'Content-Type': 'application/json'}

    # Convert request strings to JSON format
    def convert_to_json(request_str):
        try:
            return json.loads(request_str)
        except json.JSONDecodeError as e:
            if verbose:
                print(f"Error in converting string to JSON: {e}")
            return None

    # Function to send a request to the GraphHopper API
    def send_request(request_data):
        if request_data is None:
            if verbose:
                print("Invalid request data. Skipping request.")
            return None

        if verbose:
            print("Sending request to GraphHopper API...")
        response = requests.post(
            api_endpoint, headers=headers, json=request_data, params={'key': api_key})
        if response.status_code == 200:
            if verbose:
                print("Response received successfully.")
            return response.json()
        else:
            if verbose:
                print(
                    f"Failed to receive response: {response.status_code}, Response: {response.text}")
            return None

    # Sending requests and updating the DataFrame
    car_req_1_json = convert_to_json(df.iloc[row_index]["car_req_1"])
    car_req_alt_json = convert_to_json(df.iloc[row_index]["car_req_alt"])

    df.at[row_index, 'car_resp_1'] = send_request(car_req_1_json)
    df.at[row_index, 'car_resp_alt'] = send_request(car_req_alt_json)

    if verbose:
        print("Requests completed.")

In [73]:
import requests

# Define the GraphHopper API endpoint and API key
api_endpoint = "https://graphhopper.com/api/1/route"
# Replace with your actual API key
api_key = "f0b121e6-90d8-4151-b56d-25563fa4fbf9"

for i in range(4):
    call_graphhopper_api(df_routes, i,
                         api_key, verbose=True)
    print(i)

Sending request to GraphHopper API...
Response received successfully.
Sending request to GraphHopper API...
Response received successfully.
Requests completed.
0
Sending request to GraphHopper API...
Response received successfully.
Sending request to GraphHopper API...
Response received successfully.
Requests completed.
1
Sending request to GraphHopper API...
Response received successfully.
Sending request to GraphHopper API...
Response received successfully.
Requests completed.
2
Sending request to GraphHopper API...
Response received successfully.
Sending request to GraphHopper API...
Response received successfully.
Requests completed.
3


In [74]:
df_routes

,ID_mosta,Okres,br_lat,br_long,start_lat,start_long,end_lat,end_long,survey_ID,Hexagon_Coordinates,car_req_1,car_req_alt,car_resp_1,car_resp_alt
0,M6900,Prievidza,48.676146,18.519047,48.684675,18.524662,48.643682,18.487954,91460,"[[18.51931908714645, 48.676146], [18.519183043...","{\n ""points"": [\n [\n 18.524662,\n ...","{\n ""points"": [\n [\n 18.524662,\n ...","{'hints': {'visited_nodes.sum': 204, 'visited_...","{'hints': {'visited_nodes.sum': 2648, 'visited..."
1,M7339,Zlaté Moravce,48.354953,18.379679,48.382640,18.397280,48.345141,18.293945,80380,"[[18.379949367626608, 48.354953], [18.37981418...","{\n ""points"": [\n [\n 18.39728,\n ...","{\n ""points"": [\n [\n 18.39728,\n ...","{'hints': {'visited_nodes.sum': 664, 'visited_...","{'hints': {'visited_nodes.sum': 3510, 'visited..."
2,M7361,Turčianske Teplice,48.838050,18.879447,48.844674,18.933007,48.839538,18.869712,91500,"[[18.879719965489723, 48.83805], [18.879583482...","{\n ""points"": [\n [\n 18.933007,\n ...","{\n ""points"": [\n [\n 18.933007,\n ...","{'hints': {'visited_nodes.sum': 76, 'visited_n...","{'hints': {'visited_nodes.sum': 454, 'visited_..."
3,M3011,Bratislava I,48.162426,17.097242,48.170121,17.073916,48.154463,17.106079,80122,"[[17.09751135134314, 48.162426], [17.097376675...","{\n ""points"": [\n [\n 17.073916,\n ...","{\n ""points"": [\n [\n 17.073916,\n ...","{'hints': {'visited_nodes.sum': 108, 'visited_...","{'hints': {'visited_nodes.sum': 1962, 'visited..."


In [75]:
import pandas as pd

# Specify the path to your Excel file
excel_file_path = r'C:\Users\Doktor\Documents\GitHub\Bridges\Data\routes_selected_new.xlsx'
survey_file_path = r'C:\Users\Doktor\Documents\GitHub\Bridges\Data\surveys\Survey.csv'
bridges_path = r'C:\Users\Doktor\Documents\GitHub\Bridges\Data\Bridges_processed.csv'
# Load the Excel file into a DataFrame
df = pd.read_excel(excel_file_path)
df_survey = pd.read_csv(survey_file_path, encoding='utf-8', delimiter=',')
df_bridges_all = pd.read_csv(bridges_path, encoding='utf-8', delimiter=',')

In [78]:
# Merge df_survey with df_routes based on 'ID_úseku' and 'CESTA'
df_routes = df_routes.merge(
    df_survey[['ÚSEK', 'T', 'O', 'M']], left_on='survey_ID', right_on='ÚSEK', how='left')

# Drop the 'CESTA' column from the merged DataFrame, as it's redundant
df_routes.drop(columns=['ÚSEK'], inplace=True)

In [80]:
# Specify the columns you want to add from df_bridges_all
columns_to_add = ['Trieda_PK', 'Názov_mosta', 'Druh_konštrukcie', 'Material',
                  'Predmet_premostenia', 'Rok_postavenia', 'Dĺžka_premostenia',
                  'Volná_šírka_mosta', 'n_2012', 'n_2022', 'n_2023']

# Merge df_routes with df_bridges_all based on 'ID_mosta'
df_routes = df_routes.merge(df_bridges_all[columns_to_add + ['ID_mosta']],
                            on='ID_mosta', how='left')

In [82]:
def calculate_car_time_1(row):
    if row is not None:
        return row['paths'][0]['time'] / 1000 / 60
    else:
        return None


def calculate_car_time_alt(row):
    if row is not None:
        return row['paths'][0]['time'] / 1000 / 60
    else:
        return None


def calculate_car_dist_1(row):
    if row is not None:
        return row['paths'][0]['distance'] / 1000
    else:
        return None


def calculate_car_dist_alt(row):
    if row is not None:
        return row['paths'][0]['distance'] / 1000
    else:
        return None


# Calculate and store car_time_1, car_time_alt, car_dist_1, and car_dist_alt
df_routes['car_time_1'] = df_routes['car_resp_1'].apply(calculate_car_time_1)
df_routes['car_time_alt'] = df_routes['car_resp_alt'].apply(
    calculate_car_time_alt)
df_routes['car_dist_1'] = df_routes['car_resp_1'].apply(calculate_car_dist_1)
df_routes['car_dist_alt'] = df_routes['car_resp_alt'].apply(
    calculate_car_dist_alt)

In [84]:
df_routes['time_diff'] = df_routes['car_time_alt'] - df_routes['car_time_1']
df_routes['dist_diff'] = df_routes['car_dist_alt'] - df_routes['car_dist_1']

In [85]:
df_routes

,ID_mosta,Okres,br_lat,br_long,start_lat,start_long,end_lat,end_long,survey_ID,Hexagon_Coordinates,...,Volná_šírka_mosta,n_2012,n_2022,n_2023,car_time_1,car_time_alt,car_dist_1,car_dist_alt,time_diff,dist_diff
0,M6900,Prievidza,48.676146,18.519047,48.684675,18.524662,48.643682,18.487954,91460,"[[18.51931908714645, 48.676146], [18.519183043...",...,8.8,4,6,7,5.801183,19.286600,5.456028,8.504613,13.485417,3.048585
1,M7339,Zlaté Moravce,48.354953,18.379679,48.382640,18.397280,48.345141,18.293945,80380,"[[18.379949367626608, 48.354953], [18.37981418...",...,10.35,3,5,5,9.039517,17.217383,11.407774,12.823172,8.177867,1.415398
2,M7361,Turčianske Teplice,48.838050,18.879447,48.844674,18.933007,48.839538,18.869712,91500,"[[18.879719965489723, 48.83805], [18.879583482...",...,9.5,3,5,5,3.721033,9.568317,5.086734,9.917488,5.847283,4.830754
3,M3011,Bratislava I,48.162426,17.097242,48.170121,17.073916,48.154463,17.106079,80122,"[[17.09751135134314, 48.162426], [17.097376675...",...,12.75,4,5,5,4.505050,8.791950,3.186467,3.780451,4.286900,0.593984


In [114]:
import pandas as pd

# Specify the path to your pickle file
pickle_filename = 'df_routes.pkl'

df_routes_original = pd.read_pickle(pickle_filename)

In [115]:
df_routes_original.drop(columns=['Custom_coor'], inplace=True)
df_routes_original

,ID_mosta,Okres,br_lat,br_long,start_lat,start_long,end_lat,end_long,Hexagon_Coordinates,car_req_1,...,Volná_šírka_mosta,n_2012,n_2022,n_2023,car_time_1,car_time_alt,car_dist_1,car_dist_alt,time_diff,dist_diff
0,M1979,Skalica,48.757287,17.079098,48.729199,17.048036,48.802668,17.162506,"[[17.079370628365783, 48.757287384], [17.07923...","{\n ""points"": [\n [\n 17.04803577,\n ...",...,9.5,3,6,6,11.433033,18.948467,12.199233,18.500678,7.515433,6.301445
1,M1833,Senica,48.634732,16.994971,48.586200,16.991340,48.643110,17.006333,"[[16.995242375727763, 48.634732118], [16.99510...","{\n ""points"": [\n [\n 16.99134,\n ...",...,11.0,5,6,6,6.082150,11.664683,6.898138,8.519189,5.582533,1.621051
2,M3446,Bratislava IV,48.241370,17.034921,48.205450,17.042718,48.252092,17.033278,"[[17.035190832766478, 48.241369836], [17.03505...","{\n ""points"": [\n [\n 17.04271821,\n ...",...,11.22,5,5,5,7.137467,7.748417,5.345132,5.679966,0.610950,0.334834
3,M7099,Bratislava IV,48.189060,17.047200,48.179992,17.059569,48.204150,17.043150,"[[17.047469491298028, 48.18906], [17.047334745...","{\n ""points"": [\n [\n 17.059569,\n ...",...,14.0,5,5,5,5.300717,7.567200,3.173784,3.300311,2.266483,0.126527
4,M5333,Bratislava V,48.125991,17.131041,48.101220,17.121390,48.129510,17.130100,"[[17.1313097892143, 48.125990691], [17.1311752...","{\n ""points"": [\n [\n 17.12139,\n ...",...,10.2,6,6,6,3.931800,6.252917,3.500090,4.418428,2.321117,0.918338
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
143,M6688,Gelnica,48.848681,20.932527,48.860990,20.905026,48.868673,20.965148,"[[20.932800287440568, 48.848681353], [20.93266...","{\n ""points"": [\n [\n 20.905026,\n ...",...,11.5,4,6,6,8.763633,10.843683,6.087601,6.052665,2.080050,-0.034936
144,M917,Trebišov,48.635651,21.717199,48.610569,21.780682,48.699365,21.707229,"[[21.717470928681387, 48.635651368], [21.71733...","{\n ""points"": [\n [\n 21.780682,\n ...",...,14.5,3,6,6,18.262417,25.161017,15.347949,15.060859,6.898600,-0.287090
145,M4021,Vranov nad Topľou,48.985932,21.700834,48.948542,21.710467,49.056025,21.601433,"[[21.701107358648898, 48.985932049], [21.70097...","{\n ""points"": [\n [\n 21.710467,\n ...",...,9.5,5,5,6,21.240017,30.030783,18.880812,18.840312,8.790767,-0.040500
146,M1933,Myjava,48.767957,17.562174,48.785850,17.563592,48.749000,17.566630,"[[17.562446945268285, 48.767957281], [17.56231...","{\n ""points"": [\n [\n 17.563592,\n ...",...,12.5,3,6,6,9.741033,10.221417,4.300715,6.890124,0.480383,2.589409


In [116]:
df_routes_original.columns

Index(['ID_mosta', 'Okres', 'br_lat', 'br_long', 'start_lat', 'start_long',
       'end_lat', 'end_long', 'Hexagon_Coordinates', 'car_req_1',
       'car_req_alt', 'car_resp_1', 'car_resp_alt', 'ID_úseku', 'T', 'O', 'M',
       'Trieda_PK', 'Názov_mosta', 'Druh_konštrukcie', 'Material',
       'Predmet_premostenia', 'Rok_postavenia', 'Dĺžka_premostenia',
       'Volná_šírka_mosta', 'n_2012', 'n_2022', 'n_2023', 'car_time_1',
       'car_time_alt', 'car_dist_1', 'car_dist_alt', 'time_diff', 'dist_diff'],
      dtype='object')

In [117]:
# Rename the "survey_ID" column to "ID_úseku"
df_routes = df_routes.rename(columns={'survey_ID': 'ID_úseku'})

df_routes.columns

Index(['ID_mosta', 'Okres', 'br_lat', 'br_long', 'start_lat', 'start_long',
       'end_lat', 'end_long', 'ID_úseku', 'Hexagon_Coordinates', 'car_req_1',
       'car_req_alt', 'car_resp_1', 'car_resp_alt', 'T', 'O', 'M', 'Trieda_PK',
       'Názov_mosta', 'Druh_konštrukcie', 'Material', 'Predmet_premostenia',
       'Rok_postavenia', 'Dĺžka_premostenia', 'Volná_šírka_mosta', 'n_2012',
       'n_2022', 'n_2023', 'car_time_1', 'car_time_alt', 'car_dist_1',
       'car_dist_alt', 'time_diff', 'dist_diff'],
      dtype='object')

In [118]:
# Define bridge replacements
bridge_replacements = {
    'M5910': 'M6900',
    'M7339': 'M7339',  # Note: This is a replacement with the same ID
    'M3069': 'M7361',
    'M3011': 'M3011'   # Note: This is a replacement with the same ID
}

# Perform the replacements
for bridge_to_replace, bridge_replace_by in bridge_replacements.items():
    # Find the row in df_routes_original that needs to be replaced
    index_to_replace = df_routes_original[df_routes_original['ID_mosta']
                                          == bridge_to_replace].index

    # Find the replacement row in df_routes
    replacement_row = df_routes[df_routes['ID_mosta'] == bridge_replace_by]

    # Reorder columns in the replacement row to match df_routes_original
    if not replacement_row.empty and not index_to_replace.empty:
        replacement_row = replacement_row[df_routes_original.columns]
        df_routes_original.loc[index_to_replace, :] = replacement_row.values
        print(f"Replaced bridge {bridge_to_replace} with {bridge_replace_by}.")
    else:
        print(
            f"Replacement for bridge {bridge_to_replace} with {bridge_replace_by} could not be performed.")

# Verifying the replacement (optional)
print("Replacement completed. Verifying changes:")
print(df_routes_original[df_routes_original['ID_mosta'].isin(
    bridge_replacements.values())])

Replaced bridge M5910 with M6900.
Replaced bridge M7339 with M7339.
Replaced bridge M3069 with M7361.
Replaced bridge M3011 with M3011.
Replacement completed. Verifying changes:
    ID_mosta               Okres     br_lat    br_long  start_lat  start_long  \
47     M7339       Zlaté Moravce  48.354953  18.379679  48.382640   18.397280   
90     M7361  Turčianske Teplice  48.838050  18.879447  48.844674   18.933007   
107    M3011        Bratislava I  48.162426  17.097242  48.170121   17.073916   
125    M6900           Prievidza  48.676146  18.519047  48.684675   18.524662   

       end_lat   end_long                                Hexagon_Coordinates  \
47   48.345141  18.293945  [[18.379949367626608, 48.354953], [18.37981418...   
90   48.839538  18.869712  [[18.879719965489723, 48.83805], [18.879583482...   
107  48.154463  17.106079  [[17.09751135134314, 48.162426], [17.097376675...   
125  48.643682  18.487954  [[18.51931908714645, 48.676146], [18.519183043...   

               

In [119]:
df_routes_original[df_routes_original['ID_mosta'].isin(
    bridge_replacements.values())]["ID_úseku"]

47     80380
90     91500
107    80122
125    91460
Name: ID_úseku, dtype: object

In [120]:
import pandas as pd

# Assuming df_routes_original is your DataFrame
df_routes_original.to_pickle('df_routes_fixed.pkl')